# Lab 3: Iterative methods
**Theo Puranen Åhfeldt**

# **Abstract**

The objective of this report is to implement iterative methods for solving linear and non-linear equations. The implementations are tested on randomly generated matrices. The testing indicates that the implementations are successful.

# **About the code**

In [1]:
"""This program is a template for lab reports in the course"""
"""DD2363 Methods in Scientific Computing, """
"""KTH Royal Institute of Technology, Stockholm, Sweden."""

# Copyright (C) 2020 Johan Hoffman (jhoffman@kth.se)

# This file is part of the course DD2365 Advanced Computation in Fluid Mechanics
# KTH Royal Institute of Technology, Stockholm, Sweden
#
# This is free software: you can redistribute it and/or modify
# it under the terms of the GNU Lesser General Public License as published by
# the Free Software Foundation, either version 3 of the License, or
# (at your option) any later version.

# This template is maintained by Johan Hoffman
# Please report problems to jhoffman@kth.se

'KTH Royal Institute of Technology, Stockholm, Sweden.'

# **Set up environment**

To have access to the neccessary modules you have to run this cell.

In [1]:
# Load neccessary modules.
import numpy as np

# **Introduction**

In this report various iterative methods for solving both linear and non-linear equations are implemented. In particular, the iterative search methods for solving linear systems of equations, Jacobi iteration and Gauss-Seidel iteration, are implemented. Newton's method for solving nonlinear equations on the form $f(x) = 0$ where $f$ is a continous function, is implemented for both scalars and vectors.

# **Method**

Numpy arrays are used to represent vectors and matrices. Jacobi iteration and Gauss-Seidel iteration are implemented as preconditioned Richardsson iteration and Newton's method is implemented using finite difference to approximate the derivative. Each implementation is tested by its produced solution, first that it is a solution by plugging it in, and also that it matches an already known exact solution. Since finite precision arithmetic is not perfect, tests do not check for equality but that the result is close enough to what is desired. Since the methods use a low tolerence level as a stopping criterion, the fact that they return a solution means that they have converged. 

## Jacobi iteration

We will implement Jacobi iteration using Richardson iteration with a Jacobi preconditioner. First we implement the Richardson iteration.

In [1083]:
def richardson_iteration(A, b, alpha):
    n, m = A.shape
    assert np.linalg.norm(np.identity(n) - alpha * A, ord=2) < 1, print("Convergence criteria not fullfilled")
    x = np.zeros(m)
    r = b
    while np.linalg.norm(r) > 1e-10:
        r = b - np.matmul(A, x)
        x = x + alpha * r
    return x

To implement Jacobi iteration is now a simple matter of preconditioning with the matrix $B = diag(A)^{-1}$ and choosing $\alpha = 1$.

In [59]:
def jacobi_iteration(A, b):
    B = np.diag(1./np.diag(A))
    return richardson_iteration(np.matmul(B,A), np.matmul(B,b), 1)

### Tests

We test the method on randomly generated vectors and matrices. Since the Jacobi iteration is only sure to converge if $\|I - D^{-1}A\| < 1$, where $D = diag(A)$, we only expect the method to converge when this is true. To increase the chances of generating a matrix fulfilling the criterion, the matrix is first filled with small numbers, and afterwards bigger numbers are added to the diagonal. The tests are generalized so that they can be reused for the Gauss-Seidel iteration.

In [1054]:
def verify_solver_residual(A, b, solver):
    x = solver(A, b)
    assert np.linalg.norm(np.matmul(A, x) - b) < 1e-10
    
def verify_solver_exact(A, y, solver):
    b = np.matmul(A, y)
    x = solver(A, b)
    assert np.linalg.norm(x - y) < 1e-10
    
def test_jacobi_iteration(tests, rows):
    num_runs = 0
    for _ in range(tests):
        test_mat = np.random.random((rows, rows)) / 100
        extra_diag = np.diag(np.random.random((rows,)))
        test_mat += extra_diag
        test_vec = np.random.random((rows,))
        if np.linalg.norm(np.identity(rows) - np.matmul(np.linalg.inv(np.diag(np.diag(test_mat))), test_mat), ord=2) < 1:
            verify_solver_residual(test_mat, test_vec, jacobi_iteration)
            verify_solver_exact(test_mat, test_vec, jacobi_iteration)
            num_runs += 1
    print("Ran ", num_runs, " tests.")

Ran  57  tests.


## Gauss-Seidel iteration

Using the Richardson iteration, we can achieve Gauss-Seidel iteration by preconditioning with $B = L^{-1}$, where $L$ is the lower triangular matrix obtained from the matrix $A$ by zeroing out all entries above the diagonal.

In [353]:
def gauss_seidel_iteration(A, b):
    B = np.linalg.inv(np.tril(A))
    return richardson_iteration(np.matmul(B,A), np.matmul(B,b), 1)

### Tests

We reuse the tests from the Jacobi iteration, but with our new Gauss-Seidel iteration and its convergence criterion $\|I - D^{-1}A\| < 1$ where $D = L$ as previously defined.

In [1043]:
def test_gauss_seidel_iteration(tests, rows):
    num_runs = 0
    for _ in range(tests):
        test_mat = np.random.random((rows, rows)) / 20
        extra_diag = np.diag(np.random.random((rows,)) + 0.1)
        test_mat += extra_diag
        test_vec = np.random.random((rows,))
        L = np.linalg.inv(np.linalg.inv(np.tril(test_mat)))
        if np.linalg.norm(np.identity(rows) - np.matmul(L, test_mat), ord=2) < 0.98:
            verify_solver_residual(test_mat, test_vec, gauss_seidel_iteration)
            verify_solver_exact(test_mat, test_vec, gauss_seidel_iteration)
            num_runs += 1
    print("Ran ", num_runs, " tests.")

Ran  38  tests.


## Newton's method (scalar)

Newton's method for solving scalar nonlinear equations can also be viewed as a form of Richard iteration where $\alpha = -f'(x^{(k)})^{-1}$. However, since this is a function of $x^{(k)}$, it is not possible to reuse our Richardson iteration implementation. We also need to be able to compute the derivative of $f$, which would ideally be done analytically. However, for our implementation to be able to handle an arbitrary function this is not feasable, so we will use finite difference instead.

Since convergence for Newton's method depends on the initial value of $x$ that has to be decided based on the particular function, this is added as a parameter.

In [299]:
def finite_difference(f, x, h):
    return (f(x + h) - f(x)) / h

def newtons_method_scalar(f, x0):
    x = x0
    while np.abs(f(x)) > 1e-10:
        df = finite_difference(f, x, 1e-8)
        x = x - f(x)/df
    return x

### Tests

Since it is not as trivial to generate random non-linear functions as it is to generate linear ones, we will instead test on a few hand-picked examples that are easy to verify by hand. Otherwise the testing is very similar to the previous methods.

In [304]:
def verify_newton_scalar_residual(f, x0):
    x = newtons_method_scalar(f, x0)
    assert np.abs(f(x)) < 1e-10
    
def verify_newton_scalar_exact(f, x0, y):
    x = newtons_method_scalar(f, x0)
    assert np.abs(x - y) < 1e-10
    
def test_newton_scalar():
    f1 = lambda x: (x - 5)*(x + 3)
    verify_newton_scalar_residual(f1, 4)
    verify_newton_scalar_exact(f1, 4, 5)
    verify_newton_scalar_residual(f1, -4)
    verify_newton_scalar_exact(f1, -4, -3)
    
    verify_newton_scalar_residual(np.sin, 1)
    verify_newton_scalar_exact(np.sin, 1, 0)
    verify_newton_scalar_residual(np.sin, 3)
    verify_newton_scalar_exact(np.sin, 3, np.pi)

## Newton's method (vector)

Newton's method to solve vector nonlinear equations is exactly the same as for scalars but where $\alpha = -f'(x^{(k)})^{-1}$ has been replaced by the matrix $A = -f'(x^{(k)})^{-1}$, where $f'(x^{(k)})$ is the Jacobian of $f$ evaluated at $x^{(k)}$. For the same reasons as before, we will compute the Jacobian using finite difference instead of computing it analytically.

In [156]:
def jacobian(f, x, h):
    return np.array([(f(x + dx) - f(x))/h for dx in h * np.identity(x.size)]).transpose()

Instead of computing the inverse of the Jacobian directly in the update equation $x^{(k+1)} = x^{(k)} - f'(x^{(k)})^{-1} f(x^{(k)})$, we solve the linear equation $\Delta x^{(k+1)} = x^{(k+1)} - x^{(k)} = - f'(x^{(k)})^{-1} f(x^{(k)}) \Leftrightarrow f'(x^{(k)})\Delta x^{(k+1)} = - f(x^{(k)})$.

In [157]:
def newtons_method_vector(f, x0):
    x = x0
    while np.linalg.norm(f(x)) > 1e-10:
        df = jacobian(f, x, 1e-8)
        dx = np.linalg.solve(df, -f(x))
        x = x + dx
    return x

### Tests

The tests are just like for the scalar version, just with functions in multiple variables.

In [313]:
def verify_newton_vector_residual(f, x0):
    x = newtons_method_vector(f, x0)
    assert np.linalg.norm(f(x)) < 1e-10
    
def verify_newton_vector_exact(f, x0, y):
    x = newtons_method_vector(f, x0)
    assert np.linalg.norm(x - y) < 1e-10
    
def test_newton_vector():
    f1 = lambda x: np.array([x[0]*(3*x[1] + 6), x[1]*(4 - 2*x[0])])
    verify_newton_vector_residual(f1, np.array([0.5, 0.5]))
    verify_newton_vector_exact(f1, np.array([0.5, 0.5]), np.array([0, 0]))
    verify_newton_vector_residual(f1, np.array([3, -3]))
    verify_newton_vector_exact(f1, np.array([3, -3]), np.array([2, -2]))

# **Results**

The following code cell runs all test and shows that the implementions are successful:

In [1081]:
test_jacobi_iteration(100,20)
test_gauss_seidel_iteration(100,20)
test_newton_scalar()
test_newton_vector()
print("OK! completed all tests")

Ran  59  tests.
Ran  36  tests.
OK! completed all tests


# **Discussion**

The tests are comprehensive enough to give a clear indication that the implementations are correct. The fact that the methods produce a solution means 